In [53]:
# ! pip install faker
import numpy as np
import pandas as pd
import uuid
import datetime

from faker.providers.person.en import Provider
from faker import Faker

from faker import Faker
fake = Faker()
Faker.seed(0)

n_employees = 50
first_names = pd.Series(list(set(Provider.first_names))).sample(n_employees)
df_employees = pd.DataFrame(columns=["id", "name", "salary"])
df_employees["name"] = (first_names).sample(frac=1).values
mu = 28_000
minimum = 16_000
df_employees["salary"] = (np.random.pareto(a=6, size=(50,1)) * mu) + minimum
df_employees["salary"] = (df_employees["salary"]/100).astype(int) * 100
df_employees["id"] = pd.Series([str(uuid.uuid4()).split('-')[1] for x in range(n_employees)])

def create_random_project(number_of_employees, duration, year, is_ongoing=False):
    df_projects = pd.DataFrame(index=range(number_of_employees), columns=["employee_id", "project_id", "start_date", "end_date"])
    
    employees = df_employees.id.sample(number_of_employees)
    
    start_date = datetime.date(year=year, month=1, day=1)
    end_date = datetime.date(year=year+1, month=1, day=1)
    project_start_date = fake.date_between(start_date=start_date, end_date=end_date)
    project_end_date = project_start_date + datetime.timedelta(days=duration)
    
    if is_ongoing:
        project_end_date = None
        
    project_id = str(uuid.uuid4()).split('-')[4]
    
    df_projects["employee_id"] = employees.values
    df_projects["project_id"] = project_id
    df_projects["start_date"] = project_start_date
    df_projects["end_date"] = project_end_date
    
    
    return df_projects

n_projects = 150
projects = []
for proj in range(n_projects):
    number_of_employees = np.random.choice([1,2,3,4], replace=True, p=[0.25, 0.5, 0.15, 0.1], size=1)
    duration = (np.random.pareto(a=5, size=1) * 30*4) + 14
    year = np.random.choice([2017, 2018, 2019, 2020])
    is_ongoing = bool(np.random.binomial(n=1, p=0.4)) if year >= 2019 else False
    new_project = (create_random_project(number_of_employees=number_of_employees[0], duration=duration[0], year=year, is_ongoing=is_ongoing).values)
    
    projects.append(new_project)
    
projects_data = np.vstack(projects)
df_projects = (
    pd.DataFrame(projects_data, columns=["employee_id", "project_id", "start_date", "end_date"])
    .sort_values(["project_id", "start_date", "end_date"])
    .reset_index(drop=True)
)

In [54]:
df_employees.head(5)

,id,name,salary
0,8135,Linsey,19400
1,7f1c,Mikalah,16500
2,0041,Dawson,17300
3,5b10,Danyelle,40700
4,e8dd,Dixon,20900


In [55]:
df_projects.head(5)

,employee_id,project_id,start_date,end_date
0,2ff8,0077b503e9ce,2019-07-29,2019-08-29
1,c0bb,008f0a228467,2020-06-21,None
2,5a7f,00c250588ee7,2019-08-01,2019-10-25
3,c1ae,02d86fdd2535,2017-07-17,2017-09-11
4,f21c,043597d0b0bd,2020-08-21,2020-11-01


In [58]:
df=pd.merge(df_employees, df_projects, left_on='id', right_on='employee_id', how='left').drop('id', axis=1)
# df drop employees those have NOT done their projects
df = df.dropna(subset=['end_date'])
df_count=df.groupby(['employee_id'])['project_id'].count().reset_index().rename({'project_id': 'projects_counter'}, axis=1)
# df_w_p_c means df_with_projects_counter
df_w_p_c=pd.merge(df, df_count, on='employee_id',  how='left')
# df_w_p_c filtering by projects_counter >=10
df_w_p_c=df_w_p_c[df_w_p_c['projects_counter'] >= 10]
# Result sorting by salary
result=df_w_p_c[['employee_id','projects_counter','salary']].drop_duplicates().sort_values(by=['salary'])
print("Total employees those have at least 10 projects:",result.shape[0])
print('The 3 lowest paid employees those have done at least 10 projects:')
# Result apear the 3 lowest paid
print(result.head(3))
result.head(3)

Total employees those have at least 10 projects: 2
The 3 lowest paid employees those have done at least 10 projects:
    employee_id  projects_counter  salary
0          8135                11   19400
143        00e6                10   23200


,employee_id,projects_counter,salary
0,8135,11,19400
143,00e6,10,23200
